In [1]:
## Import dependencies

import numpy as np
from os import path
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim
import copy
import time


# Set default plot size
plt.rcParams["figure.figsize"] = (30,20)

# Define number of epochs used later in training
num_epochs = 5000

# Identification part of the filenames
model_base_name = '250000spec_86Metabolites_RAE_ExtendedRange_MoreLeftOut_Combined1Distribution'
base_name = '250000spec_ExtendedRange_MoreLeftOut_Combined1Distribution'    # This is the dataset base name
base_dir = '/path/to/base/directory'   # Set base directory

## Train MLP on dataset of 86 metabolites

In [2]:
# Name variable used for saving model metrics, name should reflect model used, dataset used, and other information such as # of epochs
ModelName = f"MLP_86Met_{model_base_name}Dist_TrainingAndValidation_ForManuscript_" + str(num_epochs) +"ep"

# Set the random seed
os.chdir(base_dir+'/DL-NMR-Optimization/ModelPerformanceMetrics/') 
seed = 1
torch.manual_seed(seed)
np.save(ModelName + "_Seed.npy", seed)

In [3]:
## Prepare to switch data from CPU to GPU

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    device = torch.device("cuda")          # A CUDA device object
    print("Using GPU for training.")
else:
    device = torch.device("cpu")           # A CPU object
    print("CUDA is not available. Using CPU for training.")

Using GPU for training.


In [4]:
# Switch to directory containing datasets
os.chdir(base_dir+'/DL-NMR-Optimization/GeneratedDataAndVariables')

# Load training data and max value from testing and training datasets
spectra_filename = f'Dataset86_{base_name}_ForManuscript_Spec.dat'
conc1_filename = f'Dataset86_{base_name}_ForManuscript_Conc.npy'

spectra_shape = (249996, 46000)
conc1_shape = (249996, 86)


# Load the memmap arrays
spectra_memmap = np.memmap(spectra_filename, dtype=np.float64, mode='r', shape=spectra_shape)
conc1_memmap = np.load(conc1_filename)

# Split into testing and training data
X_train_indices, X_test_indices, y_train_indices, y_test_indices = train_test_split(
    np.arange(spectra_shape[0]), np.arange(conc1_shape[0]), test_size=0.2, random_state=1
)

# Create custom dataset class
class NMRDataset(torch.utils.data.Dataset):
    def __init__(self, spectra_memmap, conc1_memmap, indices):
        self.spectra_memmap = spectra_memmap
        self.conc1_memmap = conc1_memmap
        self.indices = indices

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        actual_idx = self.indices[idx]
        X = self.spectra_memmap[actual_idx]
        y = self.conc1_memmap[actual_idx]
        return torch.tensor(X).float().to(device), torch.tensor(y).float().to(device)
    

# Create datasets
train_dataset = NMRDataset(spectra_memmap, conc1_memmap, X_train_indices)
test_dataset = NMRDataset(spectra_memmap, conc1_memmap, X_test_indices)

# Create DataLoaders
batch_size = 31  
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
## Define NN model object, define some parameters, and instantiate model

## Best params from Optuna study
#{'n_layers': 2, 
# 'activation': 'LeakyReLU', 
# 'n_units_l0': 222, 
# 'n_units_l1': 463, 
# 'learning_rate': 0.0020767074281295714, 
# 'reg_strength': 0.009375024340091184, 
# 'batch_size': 31}

# Define some model & training parameters
size_hidden1 = 222
size_hidden2 = 463
size_hidden3 = 86


# Define model
class NMR_Model_Aq(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(46000, size_hidden1)
        self.relu1 = nn.LeakyReLU()
        self.lin2 = nn.Linear(size_hidden1, size_hidden2)
        self.relu2 = nn.LeakyReLU()
        self.lin3 = nn.Linear(size_hidden2, size_hidden3)
    def forward(self, input):
        return (self.lin3(self.relu2(self.lin2(self.relu1(self.lin1(input))))))

In [9]:
class RelativeAbsoluteError(nn.Module):
    def __init__(self):
        super(RelativeAbsoluteError, self).__init__()

    def forward(self, y_pred, y_true):
        # Compute the mean of the true values
        y_mean = torch.mean(y_true)
        
        # Compute the absolute differences
        absolute_errors = torch.abs(y_true - y_pred)
        mean_absolute_errors = torch.abs(y_true - y_mean)
        
        # Compute RAE
        rae = torch.sum(absolute_errors) / torch.sum(mean_absolute_errors)
        return rae

In [10]:
from torch.cuda.amp import GradScaler, autocast

def train_and_save_best_model(model, train_loader, test_loader, num_epochs, save_path):
    criterion = RelativeAbsoluteError()
    optimizer = optim.AdamW(model.parameters(), lr = 0.0020767074281295714, weight_decay=0.009375024340091184)
    
    
    train_losses = []
    test_losses = []
    best_test_loss = float('inf')
    epochs_no_improve = 0
    patience = 50  # Set how many epochs without improvement in validation loss constitutes early stopping
    accumulation_steps = 4
    
    for epoch in range(num_epochs):
        # For timing cell run time
        start = time.time()
        model.train()
        train_loss = 0.0
        
        ## Training phase
        # Instantiate the GradScaler
        scaler = GradScaler()
        optimizer.zero_grad()  # Only zero gradients here at the start of an epoch
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            # Move data to GPU
            inputs, labels = inputs.to(device), labels.to(device)
            # Enable autocasting for forward and backward passes
            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                # Scale the loss to account for the accumulation steps
                loss = loss / accumulation_steps
            train_loss += loss.item() * inputs.size(0)
            # Scale the loss and perform backpropagation
            scaler.scale(loss).backward()

            if (batch_idx + 1) % accumulation_steps == 0:
                # Step the optimizer and update the scaler
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()  # Zero gradients after accumulation_steps

        # Testing phase
        train_losses.append(train_loss)
        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for inputs, labels in test_loader:
                # Move data to GPU
                inputs, labels = inputs.to(device), labels.to(device)
                # Enable autocasting for forward passes
                with autocast():
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                test_loss += loss.item() * inputs.size(0)
            test_losses.append(test_loss)
        
        
            
        if (epoch + 1) % 1 == 0:  # The last number here denotes how often to print loss metrics in terms of epochs
            print(f'Epoch [{epoch + 1}/{num_epochs}], '
                  f'Train Loss: {train_loss:.4f}, '
                  f'Test Loss: {test_loss:.4f}')
            
    
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            epochs_no_improve = 0
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, save_path)
        else:
            epochs_no_improve += 1
            
        if epochs_no_improve >= patience:
            print(f'Early stopping at epoch {epoch + 1}')
            break
        
        end = time.time()
        print("Epoch time: ",end-start)


    return train_losses, test_losses


def train_or_load_model(model, train_loader, test_loader, num_epochs, save_path):
    train_losses = []
    test_losses = []
    is_model_trained = False  # Initialize flag

    if os.path.isfile(save_path):
        print("Loading pretrained model from {}".format(save_path))
        checkpoint = torch.load(save_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer = optim.Adam(model.parameters())  
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        
    
    else:
        print("No pretrained model found. Training from scratch.")
        #optimizer = optim.Adam(model.parameters())  
        train_losses, test_losses = train_and_save_best_model(model, train_loader, test_loader, num_epochs, save_path)
        is_model_trained = True  # Set flag to True after training
        # Save losses per epoch
        np.save(ModelName + "_TrainLoss.npy", train_losses)
        np.save(ModelName + "_TestLoss.npy", test_losses)
    
    return train_losses, test_losses, is_model_trained  # Return the losses and flag

In [11]:
## Instantiate model and train

# For timing cell run time
start_time = time.time()


# Switch to directory for saving model parameters
os.chdir(base_dir+'/DL-NMR-Optimization/SavedParamsAndTrainingMetrics')

# Create model
model_aq = NMR_Model_Aq()

# Move the model to the GPU device
model_aq.to(device)

# Define the path to save and load the model parameters
save_path = ModelName + '_Params.pt'

# Call the function
train_losses, test_losses, is_model_trained = train_or_load_model(model_aq, train_iter, test_iter, num_epochs, save_path)


# Finish timing cell run time
end_time = time.time()
execution_time = end_time - start_time
if is_model_trained:
    np.save(ModelName + "_ExecutionTime.npy", execution_time)
    print("Execution time:", execution_time, "seconds")

No pretrained model found. Training from scratch.
Epoch [1/5000], Train Loss: 20280.9631, Test Loss: 13814.9640
Epoch time:  1463.4299216270447
Epoch [2/5000], Train Loss: 11096.0236, Test Loss: 8800.0768
Epoch time:  426.2634274959564
Epoch [3/5000], Train Loss: 6777.6097, Test Loss: 5841.2142
Epoch time:  402.8392524719238
Epoch [4/5000], Train Loss: 5022.3068, Test Loss: 4500.0571
Epoch time:  400.0149567127228
Epoch [5/5000], Train Loss: 4379.7819, Test Loss: 4433.2164
Epoch time:  399.5797176361084
Epoch [6/5000], Train Loss: 4025.9677, Test Loss: 4124.2383
Epoch time:  411.8033232688904
Epoch [7/5000], Train Loss: 3806.4347, Test Loss: 3671.3277
Epoch time:  397.2672402858734
Epoch [8/5000], Train Loss: 3637.7353, Test Loss: 3437.6496
Epoch time:  393.8951835632324
Epoch [9/5000], Train Loss: 3478.2585, Test Loss: 3487.0662
Epoch time:  395.2951440811157
Epoch [10/5000], Train Loss: 3376.7514, Test Loss: 3608.9642
Epoch time:  373.7318334579468
Epoch [11/5000], Train Loss: 3301.1

Epoch [90/5000], Train Loss: 1649.1734, Test Loss: 1597.4747
Epoch time:  383.581574678421
Epoch [91/5000], Train Loss: 1632.5195, Test Loss: 1697.0881
Epoch time:  406.80807304382324
Epoch [92/5000], Train Loss: 1634.0541, Test Loss: 1620.5070
Epoch time:  382.5830373764038
Epoch [93/5000], Train Loss: 1634.6224, Test Loss: 1765.2272
Epoch time:  372.61725211143494
Epoch [94/5000], Train Loss: 1637.6749, Test Loss: 1664.1143
Epoch time:  383.14458084106445
Epoch [95/5000], Train Loss: 1636.5187, Test Loss: 1695.1279
Epoch time:  373.85328817367554
Epoch [96/5000], Train Loss: 1627.3193, Test Loss: 1589.7931
Epoch time:  402.3304479122162
Epoch [97/5000], Train Loss: 1623.5004, Test Loss: 1664.8143
Epoch time:  439.95149636268616
Epoch [98/5000], Train Loss: 1621.0822, Test Loss: 1771.6920
Epoch time:  381.81434178352356
Epoch [99/5000], Train Loss: 1625.3539, Test Loss: 1576.8046
Epoch time:  375.54011368751526
Epoch [100/5000], Train Loss: 1608.9835, Test Loss: 1561.1296
Epoch time: 

Epoch [178/5000], Train Loss: 1547.4355, Test Loss: 1715.1310
Epoch time:  512.7827880382538
Epoch [179/5000], Train Loss: 1554.0614, Test Loss: 1763.6040
Epoch time:  442.3045873641968
Epoch [180/5000], Train Loss: 1546.8735, Test Loss: 1566.9588
Epoch time:  420.6811249256134
Epoch [181/5000], Train Loss: 1556.3623, Test Loss: 1583.0640
Epoch time:  438.58517599105835
Epoch [182/5000], Train Loss: 1551.9638, Test Loss: 1510.1938
Epoch time:  434.41526079177856
Epoch [183/5000], Train Loss: 1552.8488, Test Loss: 1624.8265
Epoch time:  505.1919515132904
Epoch [184/5000], Train Loss: 1543.9050, Test Loss: 1555.7019
Epoch time:  435.55029463768005
Epoch [185/5000], Train Loss: 1545.4838, Test Loss: 1668.4957
Epoch time:  523.2303369045258
Epoch [186/5000], Train Loss: 1542.5043, Test Loss: 1848.3613
Epoch time:  476.24966502189636
Epoch [187/5000], Train Loss: 1544.0093, Test Loss: 1523.4948
Epoch time:  492.5786643028259
Epoch [188/5000], Train Loss: 1552.7191, Test Loss: 1472.5769
Epoc

Epoch [266/5000], Train Loss: 1516.2341, Test Loss: 1439.0764
Epoch time:  372.31919527053833
Epoch [267/5000], Train Loss: 1519.7790, Test Loss: 1627.7897
Epoch time:  386.8818738460541
Epoch [268/5000], Train Loss: 1512.8061, Test Loss: 1576.6385
Epoch time:  404.8900263309479
Epoch [269/5000], Train Loss: 1515.6377, Test Loss: 1500.6312
Epoch time:  363.0595359802246
Epoch [270/5000], Train Loss: 1512.2858, Test Loss: 1460.8320
Epoch time:  390.52900981903076
Epoch [271/5000], Train Loss: 1526.3680, Test Loss: 1553.4576
Epoch time:  392.9101400375366
Epoch [272/5000], Train Loss: 1516.2152, Test Loss: 1563.6648
Epoch time:  341.1570484638214
Epoch [273/5000], Train Loss: 1518.1740, Test Loss: 1577.6428
Epoch time:  388.4290256500244
Epoch [274/5000], Train Loss: 1518.1697, Test Loss: 1525.1345
Epoch time:  403.30230951309204
Epoch [275/5000], Train Loss: 1515.8770, Test Loss: 1536.8865
Epoch time:  347.5449163913727
Epoch [276/5000], Train Loss: 1525.0732, Test Loss: 1530.0437
Epoch

In [12]:
np.array(test_losses).min()

1401.7300172820687

In [26]:
## Load training and testing datasets, validation datasets, and representative example spectra 

# Switch to directory containing datasets
os.chdir(base_dir+'/DL-NMR-Optimization/GeneratedDataAndVariables')

# Load validation dataset
spectraVal = np.load(f'Dataset86_{base_name}_ForManuscript_Val_Spec.npy')
concVal = np.load(f'Dataset86_{base_name}_ForManuscript_Val_Conc.npy')



# Load representative validation spectra and concentrations
# Load spectra of varied concentrations (all metabolites at X-mM from 0.005mm to 20mM)
ConcSpec = np.load(f'Concentration_86met_{base_name}_ForManuscript_Spec.npy')
ConcConc = np.load(f'Concentration_86met_{base_name}_ForManuscript_Conc.npy')  
#  Load uniform concentration distribution validation spectra
UniformSpec = np.load(f'UniformDist_86met_{base_name}_ForManuscript_Spec.npy')
UniformConc = np.load(f'UniformDist_86met_{base_name}_ForManuscript_Conc.npy')  
#  Load low concentration uniform concentration distribution validation spectra
LowUniformSpec = np.load(f'LowUniformDist_86met_{base_name}_ForManuscript_Spec.npy')
LowUniformConc = np.load(f'LowUniformDist_86met_{base_name}_ForManuscript_Conc.npy')
#  Load tissue mimicking concentration distribution validation spectra
MimicTissueRangeSpec = np.load(f'MimicTissueRange_86met_{base_name}_ForManuscript_Spec.npy')
MimicTissueRangeConc = np.load(f'MimicTissueRange_86met_{base_name}_ForManuscript_Conc.npy')
#  Load liver tissue mimicking concentration distribution (high relative glucose) validation spectra
MimicTissueRangeGlucSpec = np.load(f'MimicTissueRangeGluc_86met_{base_name}_ForManuscript_Spec.npy')
MimicTissueRangeGlucConc = np.load(f'MimicTissueRangeGluc_86met_{base_name}_ForManuscript_Conc.npy')
#  Load high dynamic range #2 validation spectra
HighDynamicRange2Spec = np.load(f'HighDynRange2_86met_{base_name}_ForManuscript_Spec.npy')
HighDynamicRange2Conc = np.load(f'HighDynRange2_86met_{base_name}_ForManuscript_Conc.npy') 
#  Load varied SNR validation spectra
SNR_Spec = np.load(f'SNR_86met_{base_name}_ForManuscript_Spec.npy')
SNR_Conc = np.load(f'SNR_86met_{base_name}_ForManuscript_Conc.npy')
#  Load random singlet validation spectra
Singlet_Spec = np.load(f'Singlet_86met_{base_name}_ForManuscript_Spec.npy')
Singlet_Conc = np.load(f'Singlet_86met_{base_name}_ForManuscript_Conc.npy')
#  Load random qref checker validation spectra
QrefSensSpec = np.load(f'QrefSensitivity_86met_{base_name}_ForManuscript_Spec.npy')
QrefSensConc = np.load(f'QrefSensitivity_86met_{base_name}_ForManuscript_Conc.npy')
#  Load other validation spectra
OtherValSpectra = np.load(f'OtherVal_86met_{base_name}_ForManuscript_Spec.npy')
OtherValConc = np.load(f'OtherVal_86met_{base_name}_ForManuscript_Conc.npy')




# Move the input data to the GPU device
spectraVal = torch.tensor(spectraVal).float().to(device)   
concVal = torch.tensor(concVal).float().to(device)
ConcSpec = torch.tensor(ConcSpec).float().to(device)   
ConcConc = torch.tensor(ConcConc).float().to(device)
UniformSpec = torch.tensor(UniformSpec).float().to(device)   
UniformConc = torch.tensor(UniformConc).float().to(device)
LowUniformSpec = torch.tensor(LowUniformSpec).float().to(device)   
LowUniformConc = torch.tensor(LowUniformConc).float().to(device)
MimicTissueRangeSpec = torch.tensor(MimicTissueRangeSpec).float().to(device)   
MimicTissueRangeConc = torch.tensor(MimicTissueRangeConc).float().to(device)
MimicTissueRangeGlucSpec = torch.tensor(MimicTissueRangeGlucSpec).float().to(device)   
MimicTissueRangeGlucConc = torch.tensor(MimicTissueRangeGlucConc).float().to(device)
HighDynamicRange2Spec = torch.tensor(HighDynamicRange2Spec).float().to(device)   
HighDynamicRange2Conc = torch.tensor(HighDynamicRange2Conc).float().to(device)
SNR_Spec = torch.tensor(SNR_Spec).float().to(device)   
SNR_Conc = torch.tensor(SNR_Conc).float().to(device)
Singlet_Spec = torch.tensor(Singlet_Spec).float().to(device)   
Singlet_Conc = torch.tensor(Singlet_Conc).float().to(device)
QrefSensSpec = torch.tensor(QrefSensSpec).float().to(device)   
QrefSensConc = torch.tensor(QrefSensConc).float().to(device)
OtherValSpectra = torch.tensor(OtherValSpectra).float().to(device)   
OtherValConc = torch.tensor(OtherValConc).float().to(device)

In [24]:
## Make sure best parameters are being utilized

# Switch to directory for saving model parameters
os.chdir(base_dir+'/DL-NMR-Optimization/SavedParamsAndTrainingMetrics')

# Define the path where you saved your model parameters
save_path = ModelName + '_Params.pt'

# Load the entire dictionary from the saved file
checkpoint = torch.load(save_path)

# Instantiate the model
model_aq = NMR_Model_Aq()

# Load the model's state dictionary from the loaded dictionary
model_aq.load_state_dict(checkpoint['model_state_dict'])

# Move the model to the GPU 
model_aq.to(device)

NMR_Model_Aq(
  (lin1): Linear(in_features=46000, out_features=222, bias=True)
  (relu1): LeakyReLU(negative_slope=0.01)
  (lin2): Linear(in_features=222, out_features=463, bias=True)
  (relu2): LeakyReLU(negative_slope=0.01)
  (lin3): Linear(in_features=463, out_features=86, bias=True)
)

In [15]:
## Compute absolute percent error statistics on validation set

APEs = []
MAPEs = []

for i in np.arange(5000):
    GroundTruth = concVal[i]
    model_aq.eval()
    Prediction = model_aq(spectraVal[i])

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(86):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err.cpu()))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "ValExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "ValExamples_MAPEs.npy", np.array(MAPEs))



print('Overall MAPE: ',np.array(MAPEs).mean())


Overall MAPE:  54.00655


In [27]:
## Compute absolute percent error statistics on concentration varied validation spectra

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = ConcConc[i]
    model_aq.eval()
    Prediction = model_aq(ConcSpec[i])

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(86):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err.cpu()))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "ConcExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "ConcExamples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2), " - Concentrations:",ConcConc[i][0].item())

Overall MAPE:  inf
--------------------
inf  - Concentrations: 0.0
64.59  - Concentrations: 0.004999999888241291
18.54  - Concentrations: 0.02500000037252903
6.28  - Concentrations: 0.10000000149011612
4.06  - Concentrations: 0.25
2.96  - Concentrations: 0.5
2.59  - Concentrations: 1.0
2.48  - Concentrations: 2.5
2.35  - Concentrations: 10.0
2.48  - Concentrations: 20.0


In [28]:
## Compute absolute percent error statistics on uniform distribution validation spectra

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = UniformConc[i]
    model_aq.eval()
    Prediction = model_aq(UniformSpec[i])

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(86):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err.cpu()))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "UniformExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "UniformExamples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2), " - Min Value:", np.round(UniformConc[i].min().item(),4), " - Mean Value:", np.round(UniformConc[i].mean().item(),1))

Overall MAPE:  7.4365363
--------------------
6.32  - Min Value: 0.2735  - Mean Value: 10.0
7.61  - Min Value: 0.2585  - Mean Value: 10.3
5.57  - Min Value: 1.1016  - Mean Value: 9.9
14.36  - Min Value: 0.1107  - Mean Value: 10.2
9.81  - Min Value: 0.099  - Mean Value: 9.7
6.13  - Min Value: 0.5492  - Mean Value: 10.5
6.23  - Min Value: 0.2504  - Mean Value: 9.4
6.15  - Min Value: 0.2381  - Mean Value: 11.0
6.18  - Min Value: 0.3283  - Mean Value: 9.9
6.01  - Min Value: 0.6191  - Mean Value: 9.6


In [29]:
## Compute absolute percent error statistics on low concentration uniform distribution validation spectra

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = LowUniformConc[i]
    model_aq.eval()
    Prediction = model_aq(LowUniformSpec[i])

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(86):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err.cpu()))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "LowUniformExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "LowUniformExamples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2), " - Min Value:", np.round(LowUniformConc[i].min().item(),4), " - Mean Value:", np.round(LowUniformConc[i].mean().item(),1))

Overall MAPE:  11.072065
--------------------
9.25  - Min Value: 0.0054  - Mean Value: 0.1
11.08  - Min Value: 0.0173  - Mean Value: 0.1
11.22  - Min Value: 0.0068  - Mean Value: 0.1
15.8  - Min Value: 0.0089  - Mean Value: 0.1
11.01  - Min Value: 0.0062  - Mean Value: 0.1
9.96  - Min Value: 0.0059  - Mean Value: 0.1
10.09  - Min Value: 0.008  - Mean Value: 0.1
10.28  - Min Value: 0.0051  - Mean Value: 0.1
10.19  - Min Value: 0.005  - Mean Value: 0.1
11.84  - Min Value: 0.0063  - Mean Value: 0.1


In [30]:
## Compute absolute percent error statistics on tissue mimicking distribution validation spectra

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = MimicTissueRangeConc[i]
    model_aq.eval()
    Prediction = model_aq(MimicTissueRangeSpec[i])

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(86):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err.cpu()))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "MimicTissueRangeExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "MimicTissueRangeExamples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2), " - Min Value:", np.round(MimicTissueRangeConc[i].min().item(),4), " - Mean Value:", np.round(MimicTissueRangeConc[i].mean().item(),1))

Overall MAPE:  36.18391
--------------------
47.19  - Min Value: 0.0062  - Mean Value: 1.3
48.98  - Min Value: 0.0066  - Mean Value: 0.7
19.76  - Min Value: 0.0127  - Mean Value: 0.6
26.03  - Min Value: 0.0092  - Mean Value: 0.8
35.02  - Min Value: 0.0055  - Mean Value: 0.6
24.24  - Min Value: 0.0097  - Mean Value: 0.8
54.56  - Min Value: 0.0063  - Mean Value: 0.9
47.85  - Min Value: 0.0066  - Mean Value: 0.7
22.9  - Min Value: 0.0055  - Mean Value: 0.7
35.3  - Min Value: 0.0163  - Mean Value: 0.9


In [31]:
## Compute absolute percent error statistics on tissue mimicking distribution validation spectra (high relative glucose concentration)

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = MimicTissueRangeGlucConc[i]
    model_aq.eval()
    Prediction = model_aq(MimicTissueRangeGlucSpec[i])

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(86):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err.cpu()))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "MimicTissueRangeGlucExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "MimicTissueRangeGlucExamples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2), " - Min Value:", np.round(MimicTissueRangeGlucConc[i].min().item(),4), " - Mean Value:", np.round(MimicTissueRangeGlucConc[i].mean().item(),1))

IndexError: index 44 is out of bounds for dimension 0 with size 44

In [ ]:
## Compute absolute percent error statistics on a further high dynamic range dataset

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = HighDynamicRange2Conc[i]
    model_aq.eval()
    Prediction = model_aq(HighDynamicRange2Spec[i])

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(86):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err.cpu()))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "HighDynamicRange2Examples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "HighDynamicRange2Examples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2), " - Min Value:", np.round(MimicTissueRangeGlucConc[i].min().item(),4), " - Mean Value:", np.round(MimicTissueRangeGlucConc[i].mean().item(),1))

In [32]:
## Compute absolute percent error statistics on a examples of varying SNR

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = 0.43
    model_aq.eval()
    Prediction = model_aq(SNR_Spec[i])

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(86):
        per_err = 100*(GroundTruth - Prediction_cpu[0][metabolite]) / GroundTruth
        APE.append(abs(per_err))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "SNR_Examples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "SNR_Examples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x53500 and 46000x222)

In [34]:
## Compute absolute percent error statistics on a examples of varying SNR

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = 0.43
    model_aq.eval()
    Prediction = model_aq(Singlet_Spec[i])

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(86):
        per_err = 100*(GroundTruth - Prediction_cpu[0][metabolite]) / GroundTruth
        APE.append(abs(per_err))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "Singlet_Examples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "Singlet_Examples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2))

Overall MAPE:  11.643129335222753
--------------------
3.06
3.21
3.34
5.2
10.58
10.73
14.37
17.76
21.07
27.11


In [35]:
## Compute absolute percent error statistics on a examples of varying SNR

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = 0.43
    model_aq.eval()
    Prediction = model_aq(QrefSensSpec[i])

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(86):
        per_err = 100*(GroundTruth - Prediction_cpu[0][metabolite]) / GroundTruth
        APE.append(abs(per_err))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "QrefSensitivity_Examples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "QrefSensitivity_Examples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2))

Overall MAPE:  30.838093632177873
--------------------
3.21
3.93
5.7
12.37
22.12
32.49
43.27
53.58
62.23
69.48


In [33]:
Pred = model_aq(OtherValSpectra[0])
Pred[0][Pred[0] < 0] = 0
print("Sinusoidal Baseline 1")
print(Pred[0])
print("___________")
print("___________")

Pred = model_aq(OtherValSpectra[1])
Pred[0][Pred[0] < 0] = 0
print("Sinusoidal Baseline 2")
print(Pred[0])
print("___________")
print("___________")

Pred = model_aq(OtherValSpectra[2])
Pred[0][Pred[0] < 0] = 0
print("HD-Range 1 - 0.01s and 20s")
print(Pred[0])

Pred = model_aq(OtherValSpectra[3])
Pred[0][Pred[0] < 0] = 0
print("HD-Range 2 - 0s and 20s")
print(Pred[0])

Sinusoidal Baseline 1
tensor([0.4591, 0.5962, 0.4011, 0.7034, 0.5582, 0.5321, 0.5435, 0.6375, 0.8087,
        0.4288, 0.4719, 0.5281, 0.6060, 1.3674, 0.0287, 0.7168, 0.7465, 0.4830,
        0.5559, 0.4217, 0.4636, 1.0352, 0.2180, 0.5676, 0.6732, 0.5323, 0.6258,
        0.2989, 0.7314, 0.4665, 0.4804, 0.4384, 0.5473, 0.3490, 0.4444, 0.4808,
        0.4974, 0.4607, 0.0212, 0.4551, 0.3859, 0.5624, 0.6423, 0.6331, 0.6855,
        0.3494, 0.3415, 0.4162, 0.6581, 0.6870, 0.5132, 0.4155, 0.5046, 0.9512,
        1.2636, 0.6128, 0.4545, 0.4334, 0.4541, 0.2549, 0.6656, 0.5092, 0.7408,
        0.4218, 0.3663, 0.5776, 0.3868, 0.6281, 0.4831, 0.3878, 0.3447, 0.5299,
        0.3252, 0.4587, 0.4387, 0.4163, 0.4163, 0.3584, 0.5016, 0.5141, 0.5509,
        0.4143, 0.3028, 0.2991, 0.4679, 0.5397], device='cuda:0',
       grad_fn=<SelectBackward0>)
___________
___________
Sinusoidal Baseline 2
tensor([0.3600, 0.5402, 0.0042, 0.6725, 0.3129, 0.4688, 0.3461, 0.1266, 0.5532,
        0.4718, 0.3278, 0.6460, 